In [1]:
import pandas as pd

train = pd.read_parquet('data/train.parquet')
display(train)

,title,author,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...
20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [2]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   20242 non-null  object
 1   author  18843 non-null  object
 2   text    20761 non-null  object
 3   label   20800 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 650.1+ KB


In [3]:
train.isnull().sum()

title      558
author    1957
text        39
label        0
dtype: int64

In [4]:
cleaned_df = train.dropna(subset=["title", "text"])
cleaned_df.info()

cleaned_df["author"] = cleaned_df["author"].fillna("Unknown")
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20203 entries, 0 to 20799
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   20203 non-null  object
 1   author  18285 non-null  object
 2   text    20203 non-null  object
 3   label   20203 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 789.2+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 20203 entries, 0 to 20799
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   20203 non-null  object
 1   author  20203 non-null  object
 2   text    20203 non-null  object
 3   label   20203 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 789.2+ KB


/tmp/ipykernel_4626/2309681875.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df["author"] = cleaned_df["author"].fillna("Unknown")


In [32]:
cleaned_df["merged_text"] = cleaned_df["title"] + " " + cleaned_df["text"]
display(cleaned_df["merged_text"])

/tmp/ipykernel_4626/1132859349.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df["merged_text"] = cleaned_df["title"] + " " + cleaned_df["text"]


0        House Dem Aide: We Didn’t Even See Comey’s Let...
1        FLYNN: Hillary Clinton, Big Woman on Campus - ...
2        Why the Truth Might Get You Fired Why the Trut...
3        15 Civilians Killed In Single US Airstrike Hav...
4        Iranian woman jailed for fictional unpublished...
                               ...                        
20795    Rapper T.I.: Trump a ’Poster Child For White S...
20796    N.F.L. Playoffs: Schedule, Matchups and Odds -...
20797    Macy’s Is Said to Receive Takeover Approach by...
20798    NATO, Russia To Hold Parallel Exercises In Bal...
20799    What Keeps the F-35 Alive   David Swanson is a...
Name: merged_text, Length: 20203, dtype: object

In [34]:
cleaned_df.loc[:, "merged_text"] = (
    cleaned_df["merged_text"]
    .str.lower()
    .replace(r'http[\w:/\.]+', ' ', regex=True)  # Remove URLs
    .replace(r"[^a-z\s'’]", " ", regex=True)      # Remove everything except lowercase letters, spaces, and apostrophes
    .replace(r'\s\s+', ' ', regex=True)         # Collapse multiple spaces
    .str.strip()                                # Remove leading/trailing spaces
)
display(cleaned_df["merged_text"])

0        house dem aide we didn’t even see comey’s lett...
1        flynn hillary clinton big woman on campus brei...
2        why the truth might get you fired why the trut...
3        civilians killed in single us airstrike have b...
4        iranian woman jailed for fictional unpublished...
                               ...                        
20795    rapper t i trump a ’poster child for white sup...
20796    n f l playoffs schedule matchups and odds the ...
20797    macy’s is said to receive takeover approach by...
20798    nato russia to hold parallel exercises in balk...
20799    what keeps the f alive david swanson is an aut...
Name: merged_text, Length: 20203, dtype: object

In [35]:
import contractions
cleaned_df["merged_text"] = cleaned_df["merged_text"].apply(
    lambda x: contractions.fix(x) if isinstance(x, str) else x
)
display(cleaned_df["merged_text"])

/tmp/ipykernel_4626/1282668482.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df["merged_text"] = cleaned_df["merged_text"].apply(


0        house dem aide we did not even see comey’s let...
1        flynn hillary clinton big woman on campus brei...
2        why the truth might get you fired why the trut...
3        civilians killed in single us airstrike have b...
4        iranian woman jailed for fictional unpublished...
                               ...                        
20795    rapper t i trump a ’poster child for white sup...
20796    n f l playoffs schedule matchups and odds the ...
20797    macy’s is said to receive takeover approach by...
20798    nato russia to hold parallel exercises in balk...
20799    what keeps the f alive david swanson is an aut...
Name: merged_text, Length: 20203, dtype: object

In [36]:
cleaned_df['merged_text'] = cleaned_df['merged_text'].replace(r"’", "", regex=True)
display(cleaned_df["merged_text"])


/tmp/ipykernel_4626/449746528.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['merged_text'] = cleaned_df['merged_text'].replace(r"’", "", regex=True)


0        house dem aide we did not even see comeys lett...
1        flynn hillary clinton big woman on campus brei...
2        why the truth might get you fired why the trut...
3        civilians killed in single us airstrike have b...
4        iranian woman jailed for fictional unpublished...
                               ...                        
20795    rapper t i trump a poster child for white supr...
20796    n f l playoffs schedule matchups and odds the ...
20797    macys is said to receive takeover approach by ...
20798    nato russia to hold parallel exercises in balk...
20799    what keeps the f alive david swanson is an aut...
Name: merged_text, Length: 20203, dtype: object

In [37]:
import spacy
from spacy.cli import download

# Download the SpaCy model if not already installed
model_name = "en_core_web_sm"

try:
    spacy_model = spacy.load(model_name)
    print(f"Successfully loaded model: {model_name}")
except OSError:
    print(f"Model '{model_name}' not found. Downloading...")
    download(model_name)
    spacy_model = spacy.load(model_name)
    print(f"Successfully downloaded and loaded model: {model_name}")

Successfully loaded model: en_core_web_sm


In [38]:
stopwords = spacy_model.Defaults.stop_words
print(stopwords)

{'thereafter', 'he', 'while', 'hence', 'using', 'various', 'being', '’ve', 'a', 'doing', 'elsewhere', 'also', 'nine', 'with', 'toward', 'hereafter', 'thence', 'bottom', '‘s', 'to', 'the', 'will', 'keep', 'it', 'that', 'around', 'our', 'last', 'again', 'moreover', 'beforehand', 'somewhere', 'go', 'why', 'everything', 'whence', 'seeming', 'become', '’re', 'themselves', 'during', 'towards', 'as', 'first', 'across', 'much', 'own', 'due', 'she', 'onto', 'every', 'be', 'into', 'whom', 'none', 'you', 'besides', 'cannot', 'for', 'sixty', 'twelve', 'beyond', 'same', 'do', "'ve", 'along', 'its', 'nothing', 'therein', 'after', 'quite', '’m', 'when', 'anyone', 'get', 'well', 'whether', 'myself', 'otherwise', 'never', 'nobody', 'hundred', 'me', 'either', '‘ll', 'latter', 'down', 'beside', '’s', 'your', 'anyway', 'out', 'even', "'m", 'already', 'upon', 'done', 'together', 'behind', 'fifty', 'put', 'each', 'noone', 'wherever', 'below', 'another', 'say', 'several', 'i', 'nevertheless', 'of', 'becomes'

In [39]:
def remove_stopwords(text):
    tokens = text.split()
    filtered_tokens = [token for token in tokens if token.lower() not in stopwords]
    return ' '.join(filtered_tokens)

cleaned_df['cleaned_text'] = cleaned_df['merged_text'].apply(remove_stopwords)
display(cleaned_df["cleaned_text"])

/tmp/ipykernel_4626/1133516218.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['cleaned_text'] = cleaned_df['merged_text'].apply(remove_stopwords)


0        house dem aide comeys letter jason chaffetz tw...
1        flynn hillary clinton big woman campus breitba...
2        truth fired truth fired october tension intell...
3        civilians killed single airstrike identified v...
4        iranian woman jailed fictional unpublished sto...
                               ...                        
20795    rapper t trump poster child white supremacy ra...
20796    n f l playoffs schedule matchups odds new york...
20797    macys said receive takeover approach hudsons b...
20798    nato russia hold parallel exercises balkans na...
20799    keeps f alive david swanson author activist jo...
Name: cleaned_text, Length: 20203, dtype: object

In [40]:
def lemmatize_text(text):
    doc = spacy_model(text)
    return ' '.join([token.lemma_ for token in doc])

cleaned_df['lemmatized_text'] = cleaned_df['merged_text'].apply(lemmatize_text)
display(cleaned_df['lemmatized_text'])

KeyboardInterrupt: 

# links

https://www.analyticsvidhya.com/blog/2021/07/detecting-fake-news-with-natural-language-processing/

https://www.analyticsvidhya.com/blog/2021/03/introduction-to-gated-recurrent-unit-gru/

https://medium.com/@saitejaponugoti/stop-words-in-nlp-5b248dadad47

https://thepythoncode.com/article/fake-news-classification-in-python

https://dl.acm.org/doi/10.1016/j.procs.2024.03.237

https://www.youtube.com/watch?app=desktop&v=rdz0UqQz5Sw&t=0s

https://medium.com/@rajvir.ahmed.shuvo/improving-fake-news-detection-a-journey-through-text-classification-c521b1509cee

https://medium.com/@carmenscartezini/fake-news-classification-tensorflow-x-pytorch-2a8f471a3713

https://towardsdatascience.com/detecting-fake-news-with-and-without-code-dd330ed449d9

https://towardsdatascience.com/turbo-charge-your-spacy-nlp-pipeline-551435b664ad



In [ ]:
from nltk.tokenize import word_tokenize